In [44]:
import java.io.File

val input = File("7.txt").readLines().let { lines ->
    val height = lines.size
    val s = lines.flatMapIndexed { y, line ->
        line.mapIndexedNotNull { x, char ->
            if (char == 'S') Pair(x, height - 1 - y) else null
        }
    }.first()

    val splitters = lines.flatMapIndexed { y, line ->
        line.mapIndexedNotNull { x, char ->
            if (char == '^') Pair(x, height - 1 - y) else null
        }
    }

    Pair(s, splitters)
}

In [51]:
tailrec fun countSplits(beams: Set<Pair<Int, Int>>, splitters: List<Pair<Int, Int>>, count: Int): Int {
    if (beams.any { it.second == 0}) return count
    val newBeams = mutableSetOf<Pair<Int, Int>>()
    var currentCount = 0
    for (beam in beams) {
        val newLocation = Pair(beam.first, beam.second - 1)
        if (newLocation in splitters) {
            val left = Pair(beam.first - 1, beam.second - 1)
            val right = Pair(beam.first + 1, beam.second - 1)
            newBeams.add(left)
            newBeams.add(right)
            currentCount++;
        } else {
            newBeams.add(newLocation)
        }
    }
    return countSplits(newBeams, splitters, count + currentCount)
}

/**
 * Could just modify countSplits to use List instead of Set, and to count number of beams
 * But it's gonna take memory and time...
 *
 * u/Lars-Kristian91 has a brillant solution: https://www.reddit.com/r/adventofcode/comments/1pgnmou/2025_day_7_lets_visualize/
 *
 * This is my implementation of it
 */
tailrec fun countBeams(beams: Array<Long>, splitters: List<Pair<Int, Int>>): Long {
    if (splitters.isEmpty()) return beams.sum()
    val head = splitters.first()
    val position = head.first
    if (beams[position] > 0) {
        val beamCount = beams[position]
        beams[position] = 0
        beams[position - 1] += beamCount
        beams[position + 1] += beamCount
    }
    return countBeams(beams, splitters.drop(1))
}

val part1 = countSplits(setOf(input.first), input.second, 0)
val part2 = countBeams(
    Array(input.second.maxOf { it.first + 2 }) { 0L }.apply { this[input.first.first] = 1L },
    input.second.sortedWith(compareByDescending<Pair<Int, Int>> { it.second }.thenBy { it.first })
)